# Imputation

This notebook contains two sections. First, in pre-processing, the raw data that was extracted from 3DStock, is organised into a form compatible with SimStock and assigns usages, wwr values etc. 

In the second section, missing values are then imputed.

:warning: Ensure you are using SimStock v0.2.0 

## Pre-processing

This section contains the preprocessing steps that get the data into the correct format for Simstock. First thing we need to do is sort out floor numbering. The data contains a row for each floor. The floors are numbered in ascending order but there are a couple of cases that need to be handled. Mezzanine levels had been (by some convention) given floor number -7 and lower ground floors are given -0.5. I propose we change -0.5 to -1 so that lower ground floors are just a basement. And I propose we simply record mezzanines as a first floor and shuffle all floors above up by 1. There are only 3 instances of -0.5 and 20 or so of -7, so the results shouldn't be very sensitive to these choices anyway. There is also one -99 that detones "a range of floors". This is a mistake in the data, so I suggest we delete that row. 

In [ ]:
import pandas as pd

# Read in the data
df = pd.read_csv("data/simstock_v2_pt2_selection_202203170624.csv")

# Delete the row where "floor" is equal to -99
df = df[df["floor"] != -99]

# Get unique scu_id values that have at least one mezzanine level (-7)
buildings_with_mezzanines = df[df["floor"] == -7]["scu_id"].unique()

# Define a function to transform the floors for buildings with mezzanines
def transform_floors_for_building(building_df):

    # Check this building does indeed have a mezzanine
    has_mezzanine = -7 in building_df["floor"].values
    if has_mezzanine:

        # Shuffle all upstairs floors up by one to make room
        building_df.loc[building_df["floor"] > 0, "floor"] += 1

        # Set mezzanine to floor 1
        building_df.loc[building_df["floor"] == -7, "floor"] = 1

    return building_df


# Apply the transformation to each building with mezzanines
for scu_id in buildings_with_mezzanines:
    transformed_building = transform_floors_for_building(
            df[df["scu_id"] == scu_id]
        )
    df.loc[df["scu_id"] == scu_id] = transformed_building

# Now we need to do the same with the -0.5 floors. Set them to -1.
# If there is also a basement, then shuffle down by 1 so it becomes
# a lower basement
# Get unique scu_id values that have at least one mezzanine level (-7)
buildings_with_lower = df[df["floor"] == -0.5]["scu_id"].unique()

# Define a function to transform the floors for buildings with lower floors
def transform_floors_for_building_with_lower(building_df):

    # Check this building does indeed have a mezzanine
    has_lower = -0.5 in building_df["floor"].values
    if has_lower:

        # Shuffle all basement floors down by one to make room
        building_df.loc[building_df["floor"] <= -1, "floor"] -= 1

        # Set mezzanine to floor 1
        building_df.loc[building_df["floor"] == -0.5, "floor"] = -1

    return building_df

# Apply the transformation to each building with lower floors
for scu_id in buildings_with_lower:
    transformed_building = transform_floors_for_building_with_lower(
            df[df["scu_id"] == scu_id]
        )
    df.loc[df["scu_id"] == scu_id] = transformed_building


So now all of the floors in df should be numbered sensibly. Now let's create another dataframe where each row corresponds to a single scu and the columns contain data for Simstock. This will be the dataframe we put into Simstock

In [ ]:
from shapely.wkt import loads

# Unique buildings
unique_scu_ids = df["scu_id"].unique()

# Create a new dataframe, where each row is a unique SCU
new_df = pd.DataFrame(index=unique_scu_ids, columns=["nofloors", "polygon", "shading", "height", "wwr", "construction", "age"])

# Populate the nofloors, geometry and shading columns
for scu_id in unique_scu_ids:

    # Select all the rows from the df that have a given scu_id
    rows_for_scu_id = df[df["scu_id"] == scu_id]

    # Calculate the count of occurrences for the current scu_id
    count = len(rows_for_scu_id)

    # Populate the "nofloors" column with the count
    new_df.loc[scu_id, "nofloors"] = count
    
    # Populate the "polygon" column with the first instance's "wkb_geometry" value
    new_df.loc[scu_id, "polygon"] = loads(rows_for_scu_id.iloc[0]["wkb_geometry"])

    # Set everything to be a non-shading object
    new_df.loc[scu_id, "shading"] = False

    new_df.loc[scu_id, "age"] = rows_for_scu_id.iloc[0]["age_ukbuildings_overall"]

We also now have height data. Lets add this in.

In [ ]:
# Read in height data
height_df = pd.read_csv("data/_select_b_scu_id_a_mean_object_height_cm_0_01_as_mean_hgt_m_a_es_202310241411.csv")

# Create a dictionary to store the mean height values for each 'scu_id'
mean_height_dict = height_df.groupby('scu_id')['mean_hgt_m'].mean().to_dict()

# Add height data to our main dataframe
new_df['height'] = new_df.index.map(mean_height_dict)

From looking at the data that there are around 145 SCU entries that are actually multi-polygons. They are not trivial multi-polygons as they contain more than one polygon (SimStock can handle trivial multi-polygons). All instances of these non-trivial multi-polygons are for purely domestic buildings, and all seem to be where there is one main building with one or more smaller buildings. Let's plot a couple of examples:

In [ ]:
# I can see from the data that the following SCUs are actually multi-polygons
scu1 = 52401019330040
scu2 = 52401020930003

# Lets plot them
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,2, figsize=(12,5))
object = new_df.loc[scu1, 'polygon']
for polyon in list(object.geoms):
     x, y = polyon.exterior.xy
     axes[0].plot(x, y)
object = new_df.loc[scu2, 'polygon']
for polyon in list(object.geoms):
     x, y = polyon.exterior.xy
     axes[1].plot(x, y)

plt.show()

These are houses with a shed(s) or garage, maybe some kind of granny annex. So lets just discard the sheds and keep only the main house. Clearly, we'll be losing cases where the shed/garage is powered or houses some kind of home hobbyist activity etc. 

For now, let's define a discard sheds function. This could potentially be incorporated into Simstock later.

In [ ]:
from shapely.geometry import Polygon

def discard_sheds(object: str) -> str:
    """
    Function to take a wkt representation of a 
    shapely object and, if it is a non-trivial
    MultiPolygon, discard all but the largest
    of the Polygons.
    """
    poly = object

    # If it is an ordinary polygon, 
    # then do nothing to it
    if isinstance(poly, Polygon):
        return poly

    # Now lets find the polygon 
    # with the largest area and 
    # discard the rest
    largest_polygon = None
    largest_area = 0.0

    # Iterate through all polygons in the MultiPolygon
    for polygon in list(poly.geoms):
        area = polygon.area
        if area > largest_area:
            largest_polygon = polygon
            largest_area = area

    if largest_polygon is not None:
        # Return the largest polygon as a separate Polygon object
        return largest_polygon
    else:
        # If no valid polygon is found, return None
        return None
    
# Now lets apply this function to the dataframe
new_df['polygon'] = new_df['polygon'].map(discard_sheds)

We now have a new dataframe where each row is a unique SCU. "nofloors", "polygon", "height", "age", and "shading" columns have been populated. We still need to populate "wwr", and "construction". We also need to add usage columns for each floor. Let's do this next, by add in 20 floor usage fields for each row. Of course not every row will have anywhere near 20 floors, so most of the entries will end up being blank.

In [ ]:
# Maximum number of floors
max_floors = new_df.nofloors.max()

# Add columns named "FLOOR_1: use", "FLOOR_2: use", etc.
# Add 20 new columns with empty values
for i in range(1, max_floors+1):
    new_df[f"FLOOR_{i}: use"] = ""

We now need to match the carb3 activities in the csv file to simstock usages. Here are the carb3 activities:

In [ ]:
carb3_actvities = df["ndtype_1_carb3_activity"].unique()
print(carb3_actvities)

And here are the uses available in the Simstock database:

In [ ]:
lights = pd.read_csv("data/settings/DB-Loads-LIGHTS.csv")
print(lights.Name.values)

Now we need to create a dictionary that maps the one to the other. Any case that doesn't have an activity will default to "Dwell".

In [ ]:
# We should all sanity check this
use_map = {
    "Workshop": "Factory",
    "Shop NEC": "Shop",
    "Store": "Shop", # Assuming this is American for shop? Or is it a warehouse?
    "Office (Inc Computer Centres) NEC": "Office",
    "Village hall, Scout hut, Guide hut": "Community",
    "State school": "Education",
    "Office (HQ / Institutional)": "Office",
    "Public House/Pub Restaurant": "Hospitality",
    "Restaurant": "Hospitality",
    "Wine bar": "Hospitality",
    "Car Showroom / Sales Site": "Shop",
    "Cafe": "Hospitality",
    "Warehouse": "Warehouse",
    "Bank/ insurance/ building society branch": "Shop",
    "Leisure centre (with swimming)": "Sport",
    "Multi-storey car parks": "Transport",
    "Hairdressing/Beauty Salon": "Shop", # Might be more similar to health?
    "Fire station": "Emergency",
    "Vehicle repair workshop": "Factory", # Or transport?
    "Gymnasium, fitness centre": "Sport",
    "Private School / College": "Education",
    "Surgery / Clinic / Health Centre": "Health",
    "Nursery, Creche, Playschool, Childcare": "Education",
    "Municipal Occupation NEC": "Education", # not sure what this is
    "Nightclub, discotheque": "Hospitality",
    "Sports Ground": "Sport"
}

# Now wrap the map in a function
def usage_mapper(use: str) -> str:
    try:
        return use_map[use]
    except KeyError:
        # When there is no listed usage, assume it is Dwell
        return "Dwell"

Now go through the SCUs and look up each or their row's activity. Run this activity through the map and use it to fill in the values of "FLOOR_1: use", "FLOOR_2: use", etc. in the new dataframe. We will also look to see if the building has any glazing percentage epc data.

In [ ]:
# Iterate over unique scu_ids in original dataframe
for scu_id in unique_scu_ids:

    # Filter rows for the current "scu_id"
    current_building = df[df["scu_id"] == scu_id]

    # Add this glazing data to the new_df
    new_df.at[scu_id, "wwr"] = current_building["d_epc_glazing_pct"].mean()

    # Iterate over the rows (floors) of the building
    # in ascending order of floor number
    current_building = current_building.sort_values(by="floor")
    i = 1
    for _, row in current_building.iterrows():

        # Access this floor's ndtype_1_carb3_activity
        use = row["ndtype_1_carb3_activity"]

        # Map it to the simplified uses
        activity = usage_mapper(use)
        
        # Put this into the corresponding floor usage column 
        # in the new df, at the row corresponding to 
        # this scu_id
        new_df.at[scu_id, f"FLOOR_{i}: use"] = activity

        # Increment floor counter
        i += 1

### Imputation

This section uses machine learning methods to impute missing values. This section also performs cross-validation of these imputation methods.

First, what percentage of buildings have glazing percentage, age and height data?

In [ ]:
percentage_with_wwr = (new_df["wwr"].count() / len(new_df)) * 100
percentage_with_age = (new_df["age"].count() / len(new_df)) * 100
percentage_with_height = (new_df["height"].count() / len(new_df)) * 100
mean_wwr = new_df.wwr.mean()
print(f"Percentage with wwr: {percentage_with_wwr}")
print(f"Mean wwr: {mean_wwr}")
print(f"Percentage with age: {percentage_with_age}")
print(f"Unique ages: {new_df.age.unique()}")
print(f"Percentage with height: {percentage_with_height}")

Only about half have glazing percentage. And there are a few instances of missing height and age data. Let's impute these missing values. Lets start by imputing the age and height data. Height data is obviously continuous, so a good option is K-nearst-neighbours (KNN) imputation. Age data is catagorical, so we need a classification algorithm; let's use a support vector machine classifier.

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor

# Extract building centroids and add them to the DataFrame as new columns
# This will help us use location for imputation
new_df["centroid_x"] = new_df["polygon"].apply(lambda geom: geom.centroid.x)
new_df["centroid_y"] = new_df["polygon"].apply(lambda geom: geom.centroid.y)

# Let's first impute height based on location 
# (there is only one missing height value) so
# we don't need to worry too much about methods here
# Train the KNN model
df_with_height = new_df.dropna(subset=['height'])
df_without_height = new_df[new_df['height'].isnull()] # This is only one value
regression_model = KNeighborsRegressor(n_neighbors=20, weights="distance")
regression_model.fit(df_with_height[["centroid_x", "centroid_y"]], df_with_height["height"])

# # Use the trained model to predict the missing "height" value
predicted_height = regression_model.predict(df_without_height[["centroid_x", "centroid_y"]])
print(predicted_height)
df_without_height = df_without_height.assign(height = predicted_height)
new_df = pd.concat([df_with_height, df_without_height])

percentage_with_height = (new_df["height"].count() / len(new_df)) * 100
print(f"Percentage with height: {percentage_with_height}")

# We will now use impute the 9% or so of age categories
# that are missing. First, let's validate the imputing method. 
# We first need to encode the string-based age data numerically
age_mapping = {"PRE-1914": 1, "1918-1939": 2, "1945-1980": 3, "POST-1980": 4}
new_df["encoded_age"] = new_df["age"].map(age_mapping)

# Also create an inverse map so we can recover the string values
inv_age_mapping = {v: k for k, v in age_mapping.items()}

# Now split the data into the chunk that has age and the chunk that doesnt
df_with_age = new_df.dropna(subset=['encoded_age'])
df_without_age = new_df[new_df['encoded_age'].isnull()]

# Create a support vector classifier
classifier = SVC()

# Define a scoring function to calculate mean absolute percentage error (MAPE)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Perform 5-fold cross-validation for the support vector machine
# Note that the negative sign below is just sign convention for
# scikit learn's implementation of 5-fold cv
features = ["centroid_x", "centroid_y", "height"]
target = "encoded_age"
classifier_scores = -cross_val_score(classifier, df_with_age[features], df_with_age[target], cv=5, scoring=mape_scorer)

# Calculate the average and standard deviation of MSE scores
average_mape = np.mean(classifier_scores)

print("SVM Age Classifier Performance in 5-fold Cross-Validation:")
print("Average MAPE:", average_mape)

This seems to work quite well so let's proceed and use the SVM to impute the missing ages.

In [ ]:
# Train the svm
classifier.fit(df_with_age[features], df_with_age[target])

# Use the trained model to predict missing age values
predicted_ages = classifier.predict(df_without_age[features])
df_without_age = df_without_age.assign(encoded_age = predicted_ages)
new_df = pd.concat([df_with_age, df_without_age])

# And recover the string values
new_df["age"] = new_df["encoded_age"].map(inv_age_mapping)

# Now check we have all age and height data
percentage_with_age = (new_df["age"].count() / len(new_df)) * 100
percentage_with_height = (new_df["height"].count() / len(new_df)) * 100
print(f"Percentage with age: {percentage_with_age}")
print(f"Unique ages: {new_df.age.unique()}")
print(f"Percentage with height: {percentage_with_height}")

So now we need to proceed to imputing the missing wwr values. There are a lot of these to impute: nearly half the data is missing. We can use height, age and location as predictors of wwr. We have many algorithms to choose from to do this. Let's choose linear regression and k-nearest-neighbours. Let's cross-validate them and see which performs the best. We will do each for both with and without location data, to see if location is a useful predictor of wwr. This is what we will do here:
- Validation of linear regression (both with and without location data)
- Validation of KNN for a range of n-neighbour values (both with and without location data)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Start by splitting the DataFrame into two parts: 
# one with missing "wwr" and one without
df_without_wwr = new_df[new_df["wwr"].isna()]
df_with_wwr = new_df[~new_df["wwr"].isna()]

# Perform 5-fold cross-validation for the linear regression
# without the location data
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
features = ["encoded_age", "height"]
target = "wwr"
lr_scores = -cross_val_score(LinearRegression(), df_with_wwr[features], df_with_wwr[target], cv=5, scoring=mape_scorer)
average_mape_lr_noloc = np.mean(lr_scores)

# Perform 5-fold cross-validation for the linear regression
# with the location data
features = ["centroid_x", "centroid_y", "encoded_age", "height"]
target = "wwr"
lr_scores = -cross_val_score(LinearRegression(), df_with_wwr[features], df_with_wwr[target], cv=5, scoring=mape_scorer)
average_mape_lr_withloc = np.mean(lr_scores)

# Now lets cross-validate KNN for a range of n-neighbour values
# Define a parameter grid with different values of n_neighbors to search
param_grid = {'n_neighbors': [i for i in range(1,50)]}

# Need to adjust the scorer for the way KNN works
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=True)

# Create a GridSearchCV object
# This will iterate over all values of the n_neighbour
# parameter and perform cross_validation using 5-fold 
# splitting on each. The splitting is only applied to 
# the section of the data that already has wwr values
grid_search = GridSearchCV(estimator=KNeighborsRegressor(weights='distance'), param_grid=param_grid, scoring=mape_scorer, cv=5)
features = ["encoded_age", "height"]
target = "wwr"
grid_search.fit(df_with_wwr[features], df_with_wwr[target]) 

# Get the validation scores (they come as a df)
knn_noloc_cv_df = pd.DataFrame(grid_search.cv_results_)

# Repeat above, this time including location data
grid_search = GridSearchCV(estimator=KNeighborsRegressor(weights='distance'), param_grid=param_grid, scoring=mape_scorer, cv=5)
features = ["centroid_x", "centroid_y", "encoded_age", "height"]
target = "wwr"
grid_search.fit(df_with_wwr[features], df_with_wwr[target])

# Get the validation scores (save as a df)
knn_withloc_cv_df = pd.DataFrame(grid_search.cv_results_)

# Print results
lowest_kknmape_noloc = min(knn_noloc_cv_df.mean_test_score.values)
lowest_kknmape_withloc = min(knn_withloc_cv_df.mean_test_score.values)
print(f"MAPE for linear regression without location data: {average_mape_lr_noloc}")
print(f"MAPE for linear regression with location data: {average_mape_lr_withloc}")
print(f"Lowest MAPE for knn without location data: {lowest_kknmape_noloc}")
print(f"Lowest MAPE for knn with location data: {lowest_kknmape_withloc}")

# Plot the results
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
axes.set_title("5-fold cross-validation of K-nearest neighbours")
axes.plot(knn_noloc_cv_df.param_n_neighbors.values, knn_noloc_cv_df.mean_test_score.values, label="KNN, no location data")
axes.plot(knn_withloc_cv_df.param_n_neighbors.values, knn_withloc_cv_df.mean_test_score.values, label="KNN, with location data")
axes.axhline(average_mape_lr_noloc, color="k", linestyle="--", label="Linear regression, no location data")
axes.legend(loc=1)
axes.set_xlabel("Nearest neighbours, $n$")
axes.set_ylabel("MAPE")

plt.tight_layout()
plt.show()

The above plot shows the average MAPE scores for KNN for increasing values of the nearest neighbours parameter $n$. At each value of $n$, the model was tested using 5-fold cross-validation. The black line represents the MAPE of linear regression with no location data. It is worth noting that linear regression with location data performed very badly. The best performer here seems to be KNN without location data with a nearest neighbours parameter of more than about 15. Both linear regression and KNN perform worse when they make use of location data. I think I might know why this is. Most of the missing WWR values are in purely non-domestic buildings. Looking at them on a map, most of these premises are in areas of the high street or in commercial estates where there is a great deal of heterogeneity between adjacent buildings. I think it is understandable that proximity is not a good predictor in these cases. For domestic buildings, such as in housing estates, I can believe that proximity would be a good predictor. However, we have the WWR data for most of these domestic buildings anyway.

We could go further and use more complicated methods. E.g. we could partition the data into geographical regions or clusters, and then have different regression models in each. Or we could come up with detailed heuristics and decision processes to assign WWR values. I don't think we should do this though. Such methods would be very hard to validate. And given the relatively small size of the data set and the large number of missing values, I don't think it is a fruitful line of work given the difficulty of it and the limited if any improvements it would offer.

So let's go ahead and impute the missing WWR values using KNN:

In [ ]:
# Train the KNN model
regression_model = KNeighborsRegressor(n_neighbors=20, weights="distance")
regression_model.fit(df_with_wwr[["height", "encoded_age"]], df_with_wwr["wwr"])

# Use the trained model to predict missing "wwr" values
predicted_wwr = regression_model.predict(df_without_wwr[["height", "encoded_age"]])
df_without_wwr = df_without_wwr.assign(wwr = predicted_wwr)
new_df = pd.concat([df_with_wwr, df_without_wwr])

Now let's check that we've imputed all missing WWR values and see what they look like.

In [ ]:
# Now check we have all age and height data
percentage_with_wwr = (new_df["wwr"].count() / len(new_df)) * 100
print(f"Percentage with wwr: {percentage_with_wwr}")

plt.hist(df_with_wwr.wwr.values, alpha=0.6, bins=30, label="Existing wwr values")
plt.hist(df_without_wwr.wwr.values, alpha=0.8, bins=30, label="Imputed wwr values")
plt.xlabel("WWR")
plt.ylabel("Count")
plt.legend(loc=1)
plt.show()


So we've imputed all missing WWR values with (from the cross-validation step earlier) an accuracy rate of around 36%. We can see above that the imputation has roughly captured the skewed distribution of the existing data.

Now we need to assign construction types. There are a lot of different ways we could go about this. I propose the following method. We use all the available data we have for a row, such as any epc entries, ages, usage types etc. to create clusters of different types of buildings. For each cluster we create a construction type. In the limit where we choose a cluster size of 1, then we will be assigning individual bespoke constructions to each and every building. On the other extreme, if we choose a very large cluster size then we will end up with only a handful of clusters that will serve as construction archetypes. We want to be somewhere in the middle in our approach. A small number of clusters will lead to a more archetypey approach, but will be simpler, whereas lots of clusters might be more realistic but very time consuming to assign constructions. We want to use as few clusters as we can without sacrificing too much complexity. 

To strike the right balance of clustering, let's use the "elbow" technique to test K-means clustering. This works by clustering the data for decreasing cluster sizes whilst recordering some metric(s) of in-cluster variance. As the cluster size decreases, we should observe some point of diminishing returns. This point will indicate a maximal cluster size that captures the complexity of the data.

Before we do this, let's split the data into purely non-domestic and everything else. This is because the purely non-domestic data has very different data entries (i.e. no EPC entries). Therefore it is best to treat this sperately.

In [ ]:
# Define a function to determine the type for each SCU
def determine_type(row):
    floorcols = row.filter(like="FLOOR_", axis=0)
    if "Dwell" not in floorcols.values:
        return "nondom"
    elif any((col != "" and col != "Dwell") for col in floorcols):
        return "mixed"
    else:
        return "dom"

# Record type in a new column
new_df["type"] = new_df.apply(determine_type, axis=1)

# Split the DataFrame into two based on the "type" column
nondom_df = new_df[new_df["type"] == "nondom"]
dom_df = new_df[new_df["type"] != "nondom"]

And let's save this data.

In [ ]:
new_df.to_csv("data/unclustered_df.csv")

Now let's cluster the domestic and mixed data. We can use all the EPC data we have, as well as height, age and location. This is a mix of categorical and numerical data. Therefore let's use the so-called k-prototypes clustering method. This is an extension to K-means that can handle a range of data type using suitable encodings. First, we'll need to pull in any useful EPC data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def get_epc_val(rows: pd.DataFrame, field_name: str) -> str:
    for i in range(len(rows)):
        strval = rows.iloc[i][field_name]
        if pd.notna(strval):
            return strval
    return pd.NA

# Pull in EPC data
epc_fields = [
    "epc_rating_current",
    "epc_rating_potential",
    "d_epc_mainht_fuel",
    "d_epc_mainht_plant",
    "d_epc_mainht_room",
    "d_epc_secondht_fuel",
    "d_epc_dhw_fuel",
    "d_epc_dhw_plant",
    "d_epc_envelope_flr_type",
    "d_epc_envelope_flr_insulation",
    "d_epc_envelope_rf_type",
    "d_epc_envelope_rf_insulation",
    "d_epc_envelope_wall_type",
    "d_epc_envelope_wall_insulation",
    "d_epc_glazing_type"
]
# for field in epc_fields:
#     dom_df[field] = "pd.NA"
df_without_wwr = df_without_wwr.assign(wwr = predicted_wwr)
for scu in dom_df.index:
    rows = df[df["scu_id"] == scu]
    for field in epc_fields:
        dom_df.at[scu, field] = get_epc_val(rows, field)

Now make sure the EPC values have sensible missing value formats.

In [ ]:
# Update missing values in the specified columns
dom_df.to_csv("data/domtest.csv")
nondom_df.to_csv("data/nondomtest.csv")

We have a few options for checking the clustering methods. I propose we split the domestic data into missing epc and existing epc. We then want to test epc imputation methods. 

In [ ]:
# Identify rows with at least one pd.NA in the epc_fields columns
na_rows = dom_df[epc_fields].apply(lambda row: any(pd.isna(value) for value in row), axis=1)

# Split the DataFrame
dom_df_without_epc = dom_df[na_rows]
dom_df_with_epc = dom_df[~na_rows]

How many have complete EPC data?

In [ ]:
print(len(dom_df_without_epc)/len(dom_df_with_epc))

So about 60% of the domestic data has full EPC values. Now let's cluster the data with epc values using K-prototypes for increasing cluster size.

In [ ]:
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    adjusted_rand_score,
    normalized_mutual_info_score
)
import warnings
warnings.filterwarnings("ignore")

# I need a function that takes my dataframe and a range of cluster numbers
# and returns, for each cluster number, the scores of that clustering
def compute_cluster_curve(
        df: pd.DataFrame, 
        fields: list,
        cluster_nums: list,
        measure: object = None,
        verbose: bool = True
        ) -> tuple:
    
    # Subset the DataFrame with the selected columns
    subset_df = df[fields]

    # Separate numerical and categorical columns
    num_cols = subset_df.select_dtypes(include=['float64', 'int64']).columns
    cat_cols = subset_df.select_dtypes(include=['object']).columns

    # Standardize numerical columns
    scaler = StandardScaler()
    subset_df[num_cols] = scaler.fit_transform(subset_df[num_cols])

    # Convert categorical columns to string type
    subset_df[cat_cols] = subset_df[cat_cols].astype(str)

    # Lists to hold the scores
    inertias = []
    if measure != None:
        measures = []

    # Iterate over the cluster values
    for i in cluster_nums:
        if verbose:
            print(f"Clustering with {i} clusters.")

        # Create k-prototypes model
        kproto = KPrototypes(n_clusters=i, init='Cao', verbose=0)

        # Fit the model
        clusters = kproto.fit_predict(subset_df.values, categorical=list(range(len(num_cols), len(subset_df.columns))))

        # Record scores
        inertias.append(kproto.cost_)
        if measure != None:
            measures.append(measure(clusters))

    if measure != None:
        return cluster_nums, inertias, measures
    return cluster_nums, inertias


# The columns we'd like to use for clustering
cluster_col_list = ["nofloors", "height", "age", "FLOOR_1: use", *epc_fields]

# Range of cluster values to try
cluster_nums = np.arange(1,35)

print(f"Number of entries = {len(dom_df_with_epc)}")

res = compute_cluster_curve(dom_df_with_epc,cluster_col_list,cluster_nums)

In [ ]:
# Plot the res data
plt.plot(res[0],res[1],"o")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.tight_layout()
plt.show()

In [ ]:
# Plot also the rate of change of the cluster inertias
def finite_difference_derivative(arr, step_size=1):
    """
    Compute the derivative of a 1D numpy array using finite differences.
    """
    # Using central difference for interior points
    arr = np.array(arr)
    derivative = (arr[2:] - arr[:-2])/(2*step_size)

    # Using forward/backward difference for boundary points
    derivative = np.concatenate(([arr[1] - arr[0]], derivative, [arr[-1] - arr[-2]])) / step_size

    return derivative

dinertia_dc = finite_difference_derivative(res[1])
plt.plot(res[0],dinertia_dc ,"o", color="m")
plt.xlabel("Number of clusters")
plt.ylabel("Rate of change of inertia")
plt.tight_layout()
plt.show()

Now define a clustering function.

In [ ]:
def cluster(df: pd.DataFrame,
            fields: list,
            cluster_num: int
            ) -> tuple[list, pd.DataFrame]:
    
    # Subset the DataFrame with the selected columns
    subset_df = df[fields]

    # Separate numerical and categorical columns
    num_cols = subset_df.select_dtypes(include=['float64', 'int64']).columns
    cat_cols = subset_df.select_dtypes(include=['object']).columns

    # Standardize numerical columns
    scaler = StandardScaler()
    subset_df[num_cols] = scaler.fit_transform(subset_df[num_cols])

    # Convert categorical columns to string type
    subset_df[cat_cols] = subset_df[cat_cols].astype(str)

    # Create k-prototypes model
    kproto = KPrototypes(n_clusters=cluster_num, init='Cao', verbose=0)

    # Fit the model
    clusters = kproto.fit_predict(subset_df.values, categorical=list(range(len(num_cols), len(subset_df.columns))))

    # Add the cluster labels back to the original DataFrame
    dom_df_with_epc['Cluster'] = clusters

    # Extract typical values for each cluster
    typical_values = pd.DataFrame(columns=cluster_col_list + ['Cluster'])
    for cluster_label in dom_df_with_epc['Cluster'].unique():
        cluster_data = dom_df_with_epc[dom_df_with_epc['Cluster'] == cluster_label]
        typical_values = typical_values.append(cluster_data[cluster_col_list].mode().iloc[0], ignore_index=True)
        typical_values.at[typical_values.index[-1], 'Cluster'] = cluster_label
    return clusters, typical_values


# The columns we'd like to use for clustering
cluster_col_list = ["nofloors", "height", "age", "FLOOR_1: use", *epc_fields]

# Now find typical values for different clusters
clusters, typical_values = cluster(dom_df_with_epc, cluster_col_list, 20)



In [ ]:
target_variable = "d_epc_envelope_rf_type"
mismatch_sum = 0
for c in set(clusters):

    # Get typical value for each cluster
    typical_epc = typical_values.iloc[c][target_variable]

    # Look up how many rows are in that clsuter
    filtered_rows = dom_df_with_epc[dom_df_with_epc['Cluster'] == c]

    # Use boolean indexing to filter rows where the column is not equal
    # to the typical value
    rows_not_equal_to_specified_string = filtered_rows[filtered_rows[target_variable] != typical_epc]

    # Count the number of rows
    num_rows_not_equal_to_specified_string = len(rows_not_equal_to_specified_string)

    mismatch_sum += num_rows_not_equal_to_specified_string

print(f"Mean mismatch of EPC = {mismatch_sum/len(dom_df_with_epc)}")

def percentage_mismatch(row):
    cluster_label = row['Cluster']
    mismatch_count = 0

    for col_name in cluster_col_list:
        typical_value = typical_values.iloc[cluster_label][col_name]
        if row[col_name] != typical_value:
            mismatch_count += 1

    total_attributes = len(cluster_col_list)
    return (mismatch_count / total_attributes)

# Apply the function to each row
dom_df_with_epc['MismatchPercentage'] = dom_df_with_epc.apply(percentage_mismatch, axis=1)
print(f"Mean disagreement = {dom_df_with_epc['MismatchPercentage'].mean()}")



Now redesign the clustering curve function tominclude the Gower distance and the mean disagreement and misassignment:

In [ ]:
intertias, entropies = res[1], res[2]
plt.plot(cluster_nums, intertias, "o")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()

---
We've now filled in all the fields for Simstock and handled MutliPolygon cases. Let's save the data.

In [ ]:
new_df.to_csv("data/processed_croydon.csv", index_label="SCU")